analysis: identify patterns in deck compositions that lead to wins
    1. aggregate dataset by deck composition
    2. compute win rates and average trophy gain for each deck
    3. compute card impact scores: how often each card appears in winning decks


dashboard:
    interactive filters on card, trophy range
    output:
        expected win rate
        suggested cards to swap to improve deck impact
        show which cards commonly appear with the chosen card

only analyzing chunk of data, need to reproduce this for all parquets

In [64]:
import pandas as pd
import pyarrow.parquet as pq
import pyarrow.dataset as ds
pd.set_option('display.max_columns', None)


In [3]:
chunk_size = 1_000_000

for i, chunk in enumerate(pd.read_csv("battles.csv", chunksize=chunk_size)):
    chunk.to_parquet(f"clash_chunk_{i}.parquet", index=False)
    print(f"Saved chunk {i}")

Saved chunk 0
Saved chunk 1
Saved chunk 2
Saved chunk 3
Saved chunk 4
Saved chunk 5
Saved chunk 6
Saved chunk 7
Saved chunk 8
Saved chunk 9
Saved chunk 10
Saved chunk 11
Saved chunk 12
Saved chunk 13
Saved chunk 14
Saved chunk 15
Saved chunk 16


In [4]:
df_chunk = pd.read_parquet('clash_chunk_0.parquet')

In [18]:
df_chunk16 = pd.read_parquet('clash_chunk_16.parquet')

In [65]:
df_chunk16.tail()

,Unnamed: 0,battleTime,arena.id,gameMode.id,average.startingTrophies,winner.tag,winner.startingTrophies,winner.trophyChange,winner.crowns,winner.kingTowerHitPoints,winner.princessTowersHitPoints,winner.clan.tag,winner.clan.badgeId,loser.tag,loser.startingTrophies,loser.trophyChange,loser.crowns,loser.kingTowerHitPoints,loser.clan.tag,loser.clan.badgeId,loser.princessTowersHitPoints,tournamentTag,winner.card1.id,winner.card1.level,winner.card2.id,winner.card2.level,winner.card3.id,winner.card3.level,winner.card4.id,winner.card4.level,winner.card5.id,winner.card5.level,winner.card6.id,winner.card6.level,winner.card7.id,winner.card7.level,winner.card8.id,winner.card8.level,winner.cards.list,winner.totalcard.level,winner.troop.count,winner.structure.count,winner.spell.count,winner.common.count,winner.rare.count,winner.epic.count,winner.legendary.count,winner.elixir.average,loser.card1.id,loser.card1.level,loser.card2.id,loser.card2.level,loser.card3.id,loser.card3.level,loser.card4.id,loser.card4.level,loser.card5.id,loser.card5.level,loser.card6.id,loser.card6.level,loser.card7.id,loser.card7.level,loser.card8.id,loser.card8.level,loser.cards.list,loser.totalcard.level,loser.troop.count,loser.structure.count,loser.spell.count,loser.common.count,loser.rare.count,loser.epic.count,loser.legendary.count,loser.elixir.average
795954,16795954,2020-12-27 00:02:00+00:00,54000050.0,72000044.0,5191.5,#2QCQL0VLV,5192.0,29.0,2.0,5832.0,"[1003, 91]",#Y0VJ8P2V,16000097.0,#9PYLQ0VYQ,5191.0,-29.0,0.0,5230.0,#2L9VLGG,16000081.0,None,NaN,28000011,13,28000007,13,26000009,13,28000012,13,26000045,13,26000049,13,26000041,13,26000000,13,"[26000000, 26000009, 26000041, 26000045, 26000...",104,5,0,3,3,0,4,1,4.000,26000006,13,26000011,13,28000007,13,26000015,13,26000029,13,26000037,13,26000025,13,28000001,13,"[26000006, 26000011, 26000015, 26000025, 26000...",104,6,0,2,1,1,4,2,4.500
795955,16795955,2020-12-27 00:02:22+00:00,54000050.0,72000044.0,5336.5,#20P229QVG,5333.0,30.0,3.0,5832.0,"[1749, 1175]",#2LGG0Q9,16000179.0,#G80VR88,5340.0,-30.0,0.0,NaN,#20999PR0,16000142.0,None,NaN,27000010,13,26000014,13,26000018,13,28000011,13,26000055,13,28000007,13,26000017,13,26000021,13,"[26000014, 26000017, 26000018, 26000021, 26000...",104,5,1,2,0,5,1,2,4.500,26000039,13,28000017,13,26000025,13,28000007,13,26000048,13,26000015,13,26000009,13,28000015,13,"[26000009, 26000015, 26000025, 26000039, 26000...",104,5,0,3,1,1,5,1,4.000
795956,16795956,2020-12-27 00:02:58+00:00,54000050.0,72000044.0,4945.5,#29RYVPPYG,4942.0,30.0,2.0,2967.0,[1890],#9GQ0LP2C,16000166.0,#9Q9UPPP2L,4949.0,-30.0,1.0,5446.0,#2PYYYRU,16000081.0,None,NaN,26000049,13,28000008,13,26000012,12,26000055,12,26000036,13,26000015,13,26000017,13,26000062,13,"[26000012, 26000015, 26000017, 26000036, 26000...",102,7,0,1,2,2,2,2,3.875,28000008,13,26000049,13,28000004,13,26000035,13,28000005,12,26000011,13,26000003,13,26000007,13,"[26000003, 26000007, 26000011, 26000035, 26000...",103,5,0,3,2,2,3,1,3.625
795957,16795957,2020-12-27 00:03:32+00:00,54000050.0,72000044.0,4211.0,#YUYLJUPYL,4214.0,29.0,3.0,3476.0,[824],#YQPVVRL9,16000163.0,#YYGJ2JQL0,4208.0,-29.0,1.0,NaN,#29P9QRPQ,16000164.0,None,NaN,26000018,10,26000017,9,26000011,10,26000006,10,26000012,10,26000062,10,26000055,10,26000027,8,"[26000006, 26000011, 26000012, 26000017, 26000...",77,8,0,0,0,3,3,2,4.500,26000033,10,26000047,11,28000010,10,26000022,11,28000003,10,26000051,11,28000005,10,26000085,7,"[26000022, 26000033, 26000047, 26000051, 26000...",80,5,0,3,2,1,2,3,5.750
795958,16795958,2020-12-27 00:03:37+00:00,54000050.0,72000044.0,4906.0,#PCG98LUPV,4892.0,32.0,1.0,5304.0,"[3346, 2941]",#89PPJRGC,16000044.0,#8CGL8VV00,4920.0,-32.0,0.0,5697.0,#2LVURY2,16000101.0,[3273],NaN,28000011,12,26000042,12,26000000,13,26000014,13,28000001,13,28000007,12,26000011,13,26000006,13,"[26000000, 26000006, 26000011, 26000014, 26000...",101,5,0,3,2,2,2,2,3.875,26000039,13,28000001,13,26000045,13,27000010,13,26000004,13,26000041,13,26000011,13,26000018,13,"[26000004, 26000011,

In [5]:
df_chunk.drop(columns=['Unnamed: 0', 'battleTime', 'gameMode.id', 
                 'winner.clan.tag', 'winner.clan.badgeId',
                 'loser.clan.tag', 'loser.clan.badgeId', 'tournamentTag'], inplace=True)


In [13]:
df_chunk.info()

<class 'pandas.core.frame.DataFrame'>
Index: 993442 entries, 0 to 999999
Data columns (total 66 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   arena.id                        993442 non-null  float64
 1   average.startingTrophies        993442 non-null  float64
 2   winner.tag                      993442 non-null  object 
 3   winner.startingTrophies         993442 non-null  float64
 4   winner.trophyChange             993442 non-null  float64
 5   winner.crowns                   993442 non-null  float64
 6   winner.kingTowerHitPoints       993442 non-null  float64
 7   winner.princessTowersHitPoints  973802 non-null  object 
 8   loser.tag                       993442 non-null  object 
 9   loser.startingTrophies          993442 non-null  float64
 10  loser.trophyChange              993442 non-null  float64
 11  loser.crowns                    993442 non-null  float64
 12  loser.kingTowerHitPoi

In [63]:
null_summary = df_chunk.isna().sum()
null_summary = null_summary[null_summary > 0]  # only show columns with nulls
print(null_summary)
#what do nulls mean for this?


winner.princessTowersHitPoints     19640
loser.kingTowerHitPoints          274647
loser.princessTowersHitPoints     510829
dtype: int64


In [66]:
# How many wins where loser.kingTowerHitPoints is null
df_chunk[df_chunk['loser.kingTowerHitPoints'].isna()]['winner.crowns'].value_counts()
#null means tower destroyed

winner.crowns
3.0    274647
Name: count, dtype: int64

In [67]:
# Check winner's princess towers
df_chunk[df_chunk['winner.princessTowersHitPoints'].isna()]['winner.crowns'].value_counts()

# Check loser's princess towers
df_chunk[df_chunk['loser.princessTowersHitPoints'].isna()]['winner.crowns'].value_counts()

#winner.princessTowersHitPoints     19640
#loser.kingTowerHitPoints          274647
#loser.princessTowersHitPoints     510829

winner.crowns
3.0    274647
2.0    236182
Name: count, dtype: int64

In [68]:
df_chunk['winner.princessTowersHitPoints'] = df_chunk['winner.princessTowersHitPoints'].fillna(0)
df_chunk['loser.kingTowerHitPoints'] = df_chunk['loser.kingTowerHitPoints'].fillna(0)
df_chunk['loser.princessTowersHitPoints'] = df_chunk['loser.princessTowersHitPoints'].fillna(0)

In [10]:
df_chunk.duplicated().sum()

np.int64(6558)

In [12]:
df_chunk = df_chunk.drop_duplicates()

In [20]:
df_chunk['loser.cards.list']

0         [26000000, 26000026, 26000030, 26000041, 27000...
1         [26000000, 26000003, 26000007, 26000011, 26000...
2         [26000011, 26000026, 26000030, 26000041, 27000...
3         [26000032, 26000040, 26000041, 26000049, 26000...
4         [26000012, 26000024, 26000045, 26000056, 26000...
                                ...                        
999995    [26000000, 26000004, 26000011, 26000012, 26000...
999996    [26000011, 26000012, 26000021, 26000024, 26000...
999997    [26000008, 26000011, 26000017, 26000021, 26000...
999998    [26000000, 26000004, 26000007, 26000012, 26000...
999999    [26000000, 26000017, 26000021, 26000035, 26000...
Name: loser.cards.list, Length: 993442, dtype: object

In [36]:
#win rate per deck
# Use the list as the deck representation
# Convert string to list
import ast
df_chunk['winner.cards.list'] = df_chunk['winner.cards.list'].apply(ast.literal_eval)
df_chunk['loser.cards.list']  = df_chunk['loser.cards.list'].apply(ast.literal_eval)
df_chunk['winner_deck'] = df_chunk['winner.cards.list'].apply(lambda x: tuple(sorted(x)))
df_chunk['loser_deck']  = df_chunk['loser.cards.list'].apply(lambda x: tuple(sorted(x)))
# Count wins and total matches
wins = df_chunk.groupby('winner_deck').size().rename('wins')
losses = df_chunk.groupby('loser_deck').size().rename('losses')
deck_stats = pd.concat([wins, losses], axis=1).fillna(0)
deck_stats['total'] = deck_stats['wins'] + deck_stats['losses']
deck_stats['win_rate'] = deck_stats['wins'] / deck_stats['total']


In [44]:
#what classifies as a good deck, 
top_decks = deck_stats.sort_values('win_rate', ascending=False).head(10)
popular_decks = deck_stats.sort_values('total', ascending=False).head(10)
underperforming = deck_stats[(deck_stats['total'] > 50) & (deck_stats['win_rate'] < 0.45)]
overperforming = deck_stats[(deck_stats['total'] > 50) & (deck_stats['win_rate'] > 0.6)]


In [45]:
overperforming

,wins,losses,total,win_rate
"(26000000, 26000001, 26000003, 26000012, 26000015, 26000041, 27000001, 28000004)",261.0,0.0,261.0,1.000000
"(26000000, 26000001, 26000003, 26000017, 26000033, 26000041, 28000008, 28000010)",74.0,0.0,74.0,1.000000
"(26000000, 26000001, 26000004, 26000007, 26000015, 26000027, 28000004, 28000008)",55.0,0.0,55.0,1.000000
"(26000000, 26000001, 26000005, 26000011, 26000018, 26000043, 28000002, 28000008)",53.0,0.0,53.0,1.000000
"(26000000, 26000001, 26000006, 26000021, 26000049, 26000055, 28000001, 28000008)",60.0,0.0,60.0,1.000000
...,...,...,...,...
"(26000048, 26000062, 26000063, 26000067, 26000068, 28000000, 28000011, 28000012)",236.0,55.0,291.0,0.810997
"(26000048, 26000062, 26000063, 26000067, 26000068, 28000000, 28000012, 28000015)",73.0,41.0,114.0,0.640351
"(26000049, 26000055, 26000056, 26000064, 27000006, 28000008, 28000014, 28000015)",95.0,0.0,95.0,1.000000
"(26000050, 26000052, 26000055, 26000057, 26000059, 26000061, 28000008, 28000009)",46.0,17.0,63.0,0.730159


In [58]:
#card impact score
from collections import Counter

# Flatten all winner cards
all_winner_cards = [card for deck in df_chunk['winner.cards.list'] for card in deck]

# Flatten all loser cards
all_loser_cards = [card for deck in df_chunk['loser.cards.list'] for card in deck]

# Combine for total occurrences
all_cards = all_winner_cards + all_loser_cards

# Make a set first so we only loop through each unique card
unique_cards = set(all_cards)

card_impact = {card: all_winner_cards.count(card)/all_cards.count(card) for card in unique_cards}

# Convert to DataFrame
card_impact_df = pd.DataFrame.from_dict(card_impact, orient='index', columns=['impact']).sort_values('impact', ascending=False)

print("Top 10 cards by impact score:")
print(card_impact_df.head(10))

Top 10 cards by impact score:
            impact
26000029  0.581380
26000080  0.575556
27000012  0.564949
28000014  0.556457
26000054  0.554666
28000016  0.551354
26000044  0.547187
26000060  0.545935
26000039  0.539156
28000015  0.538002


In [59]:
card_impact_df

,impact
26000029,0.581380
26000080,0.575556
27000012,0.564949
28000014,0.556457
26000054,0.554666
...,...
26000028,0.456015
26000022,0.455261
27000009,0.453058
28000006,0.449587


In [57]:
# -----------------------------
# Step 5: Card frequency
# -----------------------------
card_frequency = Counter(all_cards)
freq_df = pd.DataFrame.from_dict(card_frequency, orient='index', columns=['count']).sort_values('count', ascending=False)
print("Top 10 most used cards:")
print(freq_df.head(10))

Top 10 most used cards:
           count
28000008  570683
26000011  534783
28000011  533137
26000017  515234
28000000  502040
26000012  473448
28000001  416986
26000021  408614
26000055  374581
26000000  358434
